In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import scipy as sci
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 1000)

In [2]:
filepath = 'C:/Users/night/OneDrive/Documents/Out of the Park Developments/OOTP Baseball 22/saved_games/SBC.lg/import_export/csv/'

In [3]:
players = pd.read_csv(filepath + 'players.csv', low_memory = False)
batters_stats = pd.read_csv(filepath + 'players_career_batting_stats.csv')
pitchers_stats = pd.read_csv(filepath + 'players_career_pitching_stats.csv')
teams = pd.read_csv(filepath + 'teams.csv')

In [4]:
teams = teams[(teams['league_id'] == 145) & (teams['team_id'] <= 60)]
teams['team_name'] = teams['name'] + ' ' + teams['nickname']
teams = teams[['team_id','team_name']]
teams

,team_id,team_name
0,31,Carolina Swamp Foxes
1,32,Atlanta Braves
2,33,Baltimore Orioles
3,34,Boston Red Sox
4,35,Nashville Sounds
5,36,Chicago Cubs
6,37,Cincinnati Reds
7,38,Cleveland Spiders
8,39,Vancouver Rockies
9,40,Detroit Tigers


In [5]:
undrafted = players.loc[(players['draft_year'] == 0) & (players['age'] < 42), ('player_id', 'team_id', 'position', 'role', 'first_name', 'last_name')]
undrafted.head()

,player_id,team_id,position,role,first_name,last_name
0,1,603,8,0,Luis,Soto
1,2,896,1,12,Eric,Browning
3,4,34,2,0,Alfredo,Herrera
5,6,903,1,12,Arturo,Salazar
8,9,218,1,11,Ernesto,Montoya


In [6]:
batters = batters_stats.loc[players_stats['level_id'] == 1,['player_id', 'year', 'team_id','war']]
batters = batters.groupby(['player_id', 'year', 'team_id']).sum().reset_index()
batters.rename(columns={'war':'batter_war'}, inplace = True)
batters.count()

NameError: name 'players_stats' is not defined

In [7]:
pitchers = pitchers_stats.loc[players_stats['level_id'] == 1,['player_id', 'year', 'team_id','war']]
pitchers = pitchers.groupby(['player_id', 'year', 'team_id']).sum().reset_index()
pitchers.rename(columns={'war':'pitcher_war'}, inplace = True)
pitchers.count()

NameError: name 'players_stats' is not defined

In [8]:
eligible_players = pd.merge(batters,pitchers, how = 'outer', on = ('player_id', 'team_id', 'year')).fillna(0)
eligible_players['war'] = round(eligible_players['batter_war'] + eligible_players['pitcher_war'],2)
eligible_players = eligible_players[(eligible_players['team_id'] <= 60) & (eligible_players['team_id'] > 29)]
eligible_players = eligible_players[['player_id', 'year','team_id','war']]
eligible_players

NameError: name 'batters' is not defined

In [9]:
undrafted_stats = pd.merge(eligible_players, undrafted,on=('player_id','team_id')).merge(teams, on = 'team_id')

NameError: name 'eligible_players' is not defined

In [204]:
undrafted_stats['position'] = np.where(undrafted_stats['position'] == 2, 'C',
                              np.where(undrafted_stats['position'] == 3, '1B',
                              np.where(undrafted_stats['position'] == 4, '2B',
                              np.where(undrafted_stats['position'] == 5, '3B',
                              np.where(undrafted_stats['position'] == 6, 'SS',
                              np.where(undrafted_stats['position'] == 7, 'LF',
                              np.where(undrafted_stats['position'] == 8, 'CF',
                              np.where(undrafted_stats['position'] == 9, 'RF',
                              np.where(undrafted_stats['position'] == 10, 'DH',
                              np.where((undrafted_stats['position'] == 1) & (undrafted_stats['role'] == 11), 'SP',
                              np.where((undrafted_stats['position'] == 1) & (undrafted_stats['role'] == 12), 'RP',
                              np.where((undrafted_stats['position'] == 1) & (undrafted_stats['role'] == 13), 'CL',''))))))))))))

In [205]:
undrafted_stats.head(10)

,player_id,year,team_id,war,position,role,first_name,last_name,team_name
0,160,2039,53,0.06,3B,0,Esteban,Melgosa,Oakland Oaks
1,940,2035,53,2.64,SP,11,Jorge,Oropeza,Oakland Oaks
2,940,2036,53,3.23,SP,11,Jorge,Oropeza,Oakland Oaks
3,940,2037,53,4.19,SP,11,Jorge,Oropeza,Oakland Oaks
4,940,2038,53,4.96,SP,11,Jorge,Oropeza,Oakland Oaks
5,940,2039,53,1.62,SP,11,Jorge,Oropeza,Oakland Oaks
6,1939,2036,53,-0.09,RP,12,Alfredo,Nava,Oakland Oaks
7,24102,2036,53,0.55,RP,12,Jung-sik,Yeun,Oakland Oaks
8,24102,2037,53,1.56,RP,12,Jung-sik,Yeun,Oakland Oaks
9,24102,2038,53,0.95,RP,12,Jung-sik,Yeun,Oakland Oaks


In [206]:
undrafted_stats['player_name'] = undrafted_stats['first_name'] + ' ' + undrafted_stats['last_name']
final_cols=['player_name', 'team_name', 'position','war']
undrafted_stats_final = undrafted_stats[final_cols]


In [208]:
final_undrafted = undrafted_stats_final.groupby('team_name').sum().sort_values('war', ascending = False)

In [213]:
final_undrafted['Z-Score'] = round((final_undrafted['war'] - final_undrafted['war'].mean())/final_undrafted['war'].std(),2)

In [214]:
final_undrafted

,war,Z-Score
team_name,,
Cleveland Spiders,181.59,3.70
St. Louis Cardinals,120.64,2.11
Detroit Tigers,93.30,1.40
Arizona Athletics,90.51,1.32
Los Angeles Angels,77.81,0.99
Cincinnati Reds,44.37,0.12
Montreal Asahi,42.62,0.07
New York Mets,41.56,0.04
Toronto Blue Jays,40.81,0.02
